# 1. Install Packages

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import hvplot.pandas
import seaborn as sns
import matplotlib.pyplot as plt

# 2. Set up the Global Parameters

## 2.1 Filter Out Stocks By Volume

In [ ]:
# Fetch full stocks information from csv, which is downloaded from https://www.nasdaq.com/market-activity/stocks/screener

full_stocks = pd.read_csv('Resource/stocks_list.csv')
full_stocks

In [ ]:
# The Volume Distribution is positive skew

full_stocks['Volume'].hvplot.hist(bins = 100000, title = 'Volume Distribution').opts(xformatter = '%.0f')

In [ ]:
full_stocks['Volume'].median()

In [ ]:
full_stocks['Volume'].mean()

In [ ]:
# Check the data

full_stocks.info()

In [ ]:
# There is only 1 NaN in the Symbol column

full_stocks.isnull().sum()

In [ ]:
# Filter out stocks as Volume greater than median of the distribution

filter_stocks = full_stocks[full_stocks['Volume'] >= full_stocks['Volume'].quantile(0.5)]
filter_stocks

In [ ]:
# Check the filtered data again

filter_stocks.isnull().sum()

In [ ]:
# Get tickers from the DataFrame

assets = [symbol for symbol in filter_stocks['Symbol'].tolist()]
assets.sort()

start_date = '2022-01-01'
end_date = '2023-12-31'

## 2.2 Other Global Parameters

In [ ]:
# Define a function to set color based on value
def color_positive_negative(value):
    if value > 0:
        return 'green'
    elif value < 0:
        return 'red'
    else:
        return 'black'

# 3. Data Exploration

## 3.1 Data Collection and Reframe

In [ ]:
# Fetch stocks historical daily data from yfinance
stocks_daily = yf.download(assets, start = start_date, end = end_date, interval = '1D', auto_adjust = True)

In [ ]:
# Fetch Bitcoin historical daily data from yfinance
btc_daily = yf.download('BTC-USD', start = start_date, end = end_date, interval = '1D', auto_adjust = True)

In [ ]:
# Check stocks data

stocks_daily

In [ ]:
# Output the stocks data as csv
# stocks_daily.to_csv('stocks_daily.csv', index = True)

In [ ]:
#Check if there is NULL in the data
stocks_daily.isnull().sum()

In [ ]:
# Drop the non-useful information and check again

stocks_daily.drop(columns = 'Adj Close', inplace = True)
stocks_daily

In [ ]:
#Extract only the "Close" price data
stocks_close = stocks_daily['Close']
stocks_close

In [ ]:
# When fetching the data from yfinance, there is a hint warning some tickers are not loaded, so drop them all.
stocks_close.dropna(axis = 'columns', how = 'all', inplace = True)
stocks_close

In [ ]:
# If dropna directly, will lose all the data, that means some columns still have NaN 
stocks_close.dropna()

In [ ]:
# Fill all the NaN

stocks_close.fillna(method = 'ffill', inplace = True)
stocks_close.fillna(0, inplace = True)

In [ ]:
# Check again
stocks_close.dropna()

In [ ]:
# Check BTC data

btc_daily.isnull().sum()

In [ ]:
# Get the BTC close price data

btc_close = btc_daily['Close'].to_frame()
btc_close.columns = ['BTC']
btc_close

In [ ]:
# Combine stocks and BTC close price into one DataFrame

df_close = pd.concat([stocks_close, btc_close], axis = 1, join = 'inner')
df_close

## 3.2 Correlation Within All The Assets

### 3.2.1 Regular Correlation Analysis

In [ ]:
# Calculate daily returns for each asset, using log method
daily_returns = df_close.apply(np.log).diff(1).dropna(how = 'all')
daily_returns.fillna(0, inplace = True)
daily_returns

In [ ]:
# Show daily returns plot
# daily_returns.hvplot(xlabel = 'Date', ylabel = 'Daily Return', title = 'Daily Returns of Tickers', width = 900, height = 400)

In [ ]:
# Calculate the correlation matrix
correlation_matrix = daily_returns.corr()

# Display the correlation matrix
correlation_matrix

### 3.2.2 Distribution of Correlations

In [ ]:
correlation_matrix['BTC'].hvplot.hist(bins = 100, xlabel = 'Correlation', 
                                     title = "Distribution of Assets' Correlation with BTC")

In [ ]:
# Evenly divide the sample into three groups by correlations
# Filter 1 represents the lower range quantile 1/3 data

filter1_cor_matrix = correlation_matrix[correlation_matrix['BTC'] <= correlation_matrix['BTC'].quantile(1/3)]
filter1_cor_matrix = filter1_cor_matrix[filter1_cor_matrix.index]
filter1_cor_matrix

In [ ]:
# Evenly divide the sample into three groups by correlations
# Filter 2 represents the middle range quantile 1/3 data

filter2_cor_matrix = correlation_matrix[(correlation_matrix['BTC'] > correlation_matrix['BTC'].quantile(1/3)) & 
                                        (correlation_matrix['BTC'] <= correlation_matrix['BTC'].quantile(2/3))]

filter2_cor_matrix = filter2_cor_matrix[filter2_cor_matrix.index]
filter2_cor_matrix

In [ ]:
# Evenly divide the sample into three groups by correlations
# Filter 3 represents the upper range quantile 1/3 data

filter3_cor_matrix = correlation_matrix[correlation_matrix['BTC'] > correlation_matrix['BTC'].quantile(2/3)]

filter3_cor_matrix = filter3_cor_matrix[filter3_cor_matrix.index]
filter3_cor_matrix

In [ ]:
#filter_cor_matrix.hvplot.heatmap(width = 1000, height = 600, title = 'Correlation with BTC', rot = 90)

In [ ]:
# Select the top 5%

filter4_cor_matrix = correlation_matrix[correlation_matrix['BTC'] >= correlation_matrix['BTC'].quantile(0.95)]

filter4_cor_matrix = filter4_cor_matrix[filter4_cor_matrix.index]
filter4_cor_matrix

### 3.2.3 Shift Correlation Analysis

In [ ]:
"""
    Helper function to calculate the shift correlation.

    Args:
        days (int): Days of shift
        daily_rtn (DataFrame): Daily return data including stocks and BTC
        assets (List): Stocks in the portfoio
        
"""
def shift_cor(days, daily_rtn, assets):
    
    stocks_daily_rtn = daily_rtn[assets].drop(columns = 'BTC') 
    shift_daily_rtn = stocks_daily_rtn.shift(days)
    shift_daily_rtn.dropna(inplace = True)
    shift_daily_rtn.reset_index(drop = True, inplace = True)
    
    btc_daily_rtn = daily_rtn['BTC'].to_frame()
    btc_daily_rtn.reset_index(drop = True, inplace = True)
    
    cor_matrix = daily_rtn[assets].corr()
    
    shift_rtn_matrix = pd.concat([shift_daily_rtn, btc_daily_rtn], join = 'inner', axis = 1)
    shift_cor_matrix = shift_rtn_matrix.corr()
    change_cor_matrix = (shift_cor_matrix - cor_matrix) /cor_matrix * 100
    
    return change_cor_matrix['BTC'].hvplot.hist(bins = 1000, xlabel = 'Changes of Correlation with BTC', 
                                                title = f"Distribution of Changes of Correlation with BTC for Shift {days} Days", 
                                                hover_color = 'orange', rot = 90, width = 900,
                                                color = change_cor_matrix['BTC'].apply(color_positive_negative))
    

In [ ]:
# Shift 2 days for stocks daily return to see if the correlation with BTC will increase

stocks_daily_return = daily_returns[filter_cor_matrix.index].drop(columns = 'BTC')
stocks_daily_return = stocks_daily_return.shift(2)
stocks_daily_return.dropna(inplace = True)
stocks_daily_return.reset_index(drop = True, inplace = True)
stocks_daily_return

In [ ]:
# Combine the shifted stocks daily return with BTC daily return, and calculate its correlation

shift_rtn_matrix = pd.concat([stocks_daily_return, btc_daily_return], join = 'inner', axis = 1)
shift_cor_matrix = shift_rtn_matrix.corr()
shift_cor_matrix

In [ ]:
# Extract BTC daily return

btc_daily_return = daily_returns['BTC'].to_frame()
btc_daily_return.reset_index(drop = True, inplace = True)
btc_daily_return

In [ ]:
# Check if there is difference for the correlation
# if the correlation after shift is greater than the original, there is a chance BTC would be a leading indicator
filter_cor_matrix < shift_cor_matrix

In [ ]:
# The percent change of correlation
change_cor_matrix = (shift_cor_matrix - filter_cor_matrix) /filter_cor_matrix * 100
change_cor_matrix

In [ ]:
# Plot the changes of correlation

change_cor_plot = change_cor_matrix['BTC'].hvplot.bar(title = "Changes of Correlation with BTC for Shift 2 Days", 
                                                      xlabel = 'Assets', ylabel = 'Change of Correlation%', 
                                                      hover_color = 'orange', 
                                                      color = change_cor_matrix['BTC'].apply(color_positive_negative))
change_cor_plot

In [ ]:
shift_1 = shift_cor(1, daily_returns, filter3_cor_matrix.index)
shift_1

In [ ]:
shift_2 = shift_cor(2, daily_returns, filter3_cor_matrix.index)
shift_2

In [ ]:
shift_3 = shift_cor(3, daily_returns, filter3_cor_matrix.index)
shift_3

In [ ]:
shift_4 = shift_cor(4, daily_returns, filter3_cor_matrix.index)
shift_4

In [ ]:
shift_5 = shift_cor(5, daily_returns, filter3_cor_matrix.index)
shift_5

In [ ]:
shift_6 = shift_cor(6, daily_returns, filter3_cor_matrix.index)
shift_6

In [ ]:
shift_7 = shift_cor(7, daily_returns, filter3_cor_matrix.index)
shift_7

## 3.3 Changes of Correlation over time

In [ ]:
"""
    Helper function to calculate the correlation with two tickers.

    Args:
        daily_return (DataFrame): A DataFrame contains the tickers' daily returns information
        tickers (list): list of the tickers' names
        rolling_window: rolling days for the correlation
"""
def correlation(daily_return, tickers, rolling_window):
    
    # Generate a correlation matrix between tickers
    cor = daily_return[tickers].rolling(window = rolling_window).corr().dropna()
    
    # Delete Ticker Index, keep Date Index only
    df_filtered = cor.loc[cor.index.get_level_values(1) == tickers[0], :]
    df_filtered.reset_index(level = 1, inplace = True)
    
    # Drop and rename columns
    df_filtered.drop(columns = ['level_1', tickers[0]], inplace = True)
    df_filtered.columns = ['Correlation']
    
    return df_filtered

In [ ]:
# Show the correlation between AAPL and BTC over time

btc_riot_corr = correlation(daily_returns, tickers = ['AAPL', 'BTC'], rolling_window = 30)
btc_riot_corr

In [ ]:
# Plot the correlation over time

btc_riot_corr.hvplot(title = 'Correlation Over Time', xlabel = 'Days', ylabel = 'Correlation', width = 800, height = 400)

## 3.4 Betas of The Assets Compared With BTC

In [ ]:
# Calculate covariance of the sample
covariance_assets = daily_returns.cov().loc['BTC']

# Calculate variance of BTC
var_btc = daily_returns['BTC'].var()

# Computing beta
beta = covariance_assets / var_btc
beta.sort_values(ascending = False)

In [ ]:
# Plot the beta

beta.hvplot.hist(bins = 3600, title = 'Distribution of Beta relative with BTC', xlabel = 'Beta', 
                 hover_color = 'orange', width = 900)

## 3.5 Sharpe Ratio of Assets

In [ ]:
#Set the annual interest rate
interest_rate = 0.02

### 3.5.1 Filter 1

In [ ]:
# Construct the portfolio assets
portfolio_1 = filter1_cor_matrix.index.tolist()

#Calculate the annual volavility of tickers
daily_vol_1 = daily_returns[portfolio_1].std() * np.sqrt(252)

# There are some NaN in the result, drop them and re-extract the index
daily_vol_1.dropna(inplace = True)

portfolio_1 = daily_vol_1.index

#Calculate the Sharpe Ratio of tickers
avg_rtn_1 = daily_returns[portfolio_1].mean() * 252

sharpe_ratio_1 = (avg_rtn_1 - interest_rate) / daily_vol_1
sharpe_ratio_1

In [ ]:
#Show the plot of Sharpe Ratio

sharpe_ratio_1.sort_values(ascending = False).hvplot.bar(xlabel = "Tickers", ylabel = "Sharpe Ratio", 
                                                       title = "Sharpe Ratio for Filter 1", hover_color = "orange", rot = 90,
                                                       color = sharpe_ratio_1.apply(color_positive_negative)) 

### 3.5.2 Filter 2

In [ ]:
# Construct the portfolio assets
portfolio_2 = filter2_cor_matrix.index.tolist()

#Calculate the annual volavility of tickers
daily_vol_2 = daily_returns[portfolio_2].std() * np.sqrt(252)
daily_vol_2.dropna(inplace = True)

portfolio_2 = daily_vol_2.index

#Calculate the Sharpe Ratio of tickers
avg_rtn_2 = daily_returns[portfolio_2].mean() * 252
sharpe_ratio_2 = (avg_rtn_2 - interest_rate) / daily_vol_2
sharpe_ratio_2

In [ ]:
#Show the plot of Sharpe Ratio

sharpe_ratio_2.sort_values(ascending = False).hvplot.bar(xlabel = "Tickers", ylabel = "Sharpe Ratio", 
                                                       title = "Sharpe Ratio for Filter 2", hover_color = "orange", rot = 90,
                                                       color = sharpe_ratio_2.apply(color_positive_negative)) 

### 3.5.3 Filter 3

In [ ]:
# Construct the portfolio assets
portfolio_3 = filter3_cor_matrix.index.tolist()
portfolio_3.pop() # remove BTC

#Calculate the annual volavility of tickers
daily_vol_3 = daily_returns[portfolio_3].std() * np.sqrt(252)
daily_vol_3.dropna(inplace = True)

portfolio_3 = daily_vol_3.index

#Calculate the Sharpe Ratio of tickers
avg_rtn_3 = daily_returns[portfolio_3].mean() * 252
sharpe_ratio_3 = (avg_rtn_3 - interest_rate) / daily_vol_3
sharpe_ratio_3

In [ ]:
#Show the plot of Sharpe Ratio

sharpe_ratio_3.sort_values(ascending = False).hvplot.bar(xlabel = "Tickers", ylabel = "Sharpe Ratio", 
                                                       title = "Sharpe Ratio for Filter 3", hover_color = "orange", rot = 90,
                                                       color = sharpe_ratio_3.apply(color_positive_negative)) 

### 3.5.4 Filter 4

In [ ]:
# Construct the portfolio assets
portfolio_4 = filter4_cor_matrix.index.tolist()
portfolio_4.pop() # remove BTC

#Calculate the annual volavility of tickers
daily_vol_4 = daily_returns[portfolio_4].std() * np.sqrt(252)
daily_vol_4.dropna(inplace = True)

portfolio_4 = daily_vol_4.index

#Calculate the Sharpe Ratio of tickers
avg_rtn_4 = daily_returns[portfolio_4].mean() * 252
sharpe_ratio_4 = (avg_rtn_4 - interest_rate) / daily_vol_4
sharpe_ratio_4

In [ ]:
#Show the plot of Sharpe Ratio

sharpe_ratio_4.sort_values(ascending = False).hvplot.bar(xlabel = "Tickers", ylabel = "Sharpe Ratio", width = 1000,
                                                       title = "Sharpe Ratio for Filter 4", hover_color = "orange", rot = 90,
                                                       color = sharpe_ratio_4.apply(color_positive_negative)) 

### 3.5.5 Filter 5

In [ ]:
# In order to mimic BTC, select the stokcs with highest correlated and beta with BTC

filter_beta = beta[beta > beta.quantile(0.95)]

portfolio_5 = portfolio_4[portfolio_4.isin(filter_beta.index)]
portfolio_5

In [ ]:
# Filter out the positive Sharpe Ratios

sharpe_ratio_5 = sharpe_ratio_4[portfolio_5]
sharpe_ratio_5

In [ ]:
#Calculate the annual volavility of tickers

daily_vol_5 = daily_returns[portfolio_5].std() * np.sqrt(252)

#Calculate the Sharpe Ratio of tickers
avg_rtn_5 = daily_returns[portfolio_5].mean() * 252

In [ ]:
#Show the plot of Sharpe Ratio

sharpe_ratio_5.hvplot.bar(xlabel = "Tickers", ylabel = "Sharpe Ratio", width = 900,
                                                       title = "Sharpe Ratio for Filter 5", hover_color = "orange", rot = 90,
                                                       color = sharpe_ratio_4.apply(color_positive_negative)) 

# 4. Finding the efficient frontier using Monte Carlo simulations

### Set Up Common Parameters And Functions

In [ ]:
"""
    Helper function to calculate the portfolio's volatility.

    Args:
        weights (np.array): An array containing the portfolio weights
        cov_matrix (DataFrame): Covariance of tickers in portfolio
"""
def portf_vol(weights, cov_matrix):
    portf_vol = []
    for i in range(len(weights)):
        vol = np.sqrt(np.dot(weights[i].T, np.dot(cov_matrix, weights[i])))
        portf_vol.append(vol)
    portf_vol = np.array(portf_vol)
    return portf_vol

In [ ]:
#Set up the Monte Carlo Model parameters
np.random.seed(42)
n_portfolios = 10 ** 5

In [ ]:
"""
    Helper function to calculate the efficient frontier curve.

    Args:
        portf_res (DataFrame): A portfolio result comes from 4.1
        portf_rtns (DataFrame): A portfolio return comes from 4.1
        portf_vol (DataFrame): A portfolio volatility comes from 4.1
"""

def ef_curve(portf_res, portf_rtns, portf_vol):
    ef_rtn = []
    ef_vol = []
    possible_ef_rtns = np.linspace(portf_res['Returns'].min(), portf_res['Returns'].max(), 1000)
    
    possible_ef_rtns = np.round(possible_ef_rtns, 3)
    round_rtns = np.round(portf_rtns, 3)
    
    for rtn in possible_ef_rtns:
        if rtn in round_rtns:
            ef_rtn.append(rtn)
            matched_ind = np.where(round_rtns == rtn)
            ef_vol.append(np.min(portf_vol[matched_ind]))
    ef_df = pd.DataFrame({'Returns': ef_rtn, 'Volatility': ef_vol})
    return ef_df

In [ ]:
"""
    Helper function for printing the performance summary of a portfolio.

    Args:
        perf (pd.Series): Series containing the perf metrics
        weights (np.array): An array containing the portfolio weights
        assets (list): list of the asset names
        name (str): the name of the portfolio
"""

def print_portfolio_summary(perf, weights, assets, name):
    print(f"{name} portfolio ----")
    print("\nPerformance")
    for index, value in perf.items():
        print(f"{index}: {100 * value:.2f}% ", end = "", flush = True)
    print("\n\nWeights")
    for x, y in zip(assets, weights):
        print(f"{x}: {100 * y:.2f}% ", end = "\n", flush = True)
    print('Total weigths: ', 100 * weights.sum())

## 4.1 Monte Carlo Simulation 

### 4.1.1 Filter 1

In [ ]:
n_assets_1 = len(portfolio_1)

weights_1 = np.random.random(size = (n_portfolios, n_assets_1))
weights_1 /= np.sum(weights_1, axis = 1)[:, np.newaxis]

cov_matrix_1 = daily_returns[portfolio_1].cov() * 252

In [ ]:
#Calculate the portfolio matrixs
portf_rtns_1 = np.dot(weights_1, avg_rtn_1)
portf_vol_1 = portf_vol(weights_1, cov_matrix_1)
portf_sharpe_ratio_1 = (portf_rtns_1 - interest_rate) / portf_vol_1

In [ ]:
#Transform to DataFrame
portf_res_1 = pd.DataFrame({'Returns': portf_rtns_1, 'Volatility': portf_vol_1, 'Sharpe Ratio': portf_sharpe_ratio_1})
portf_res_1

### 4.1.2 Filter 2

In [ ]:
n_assets_2 = len(portfolio_2)

weights_2 = np.random.random(size = (n_portfolios, n_assets_2))
weights_2 /= np.sum(weights_2, axis = 1)[:, np.newaxis]

cov_matrix_2 = daily_returns[portfolio_2].fillna(0).cov() * 252

In [ ]:
#Calculate the portfolio matrixs
portf_rtns_2 = np.dot(weights_2, avg_rtn_2)
portf_vol_2 = portf_vol(weights_2, cov_matrix_2)
portf_sharpe_ratio_2 = (portf_rtns_2 - interest_rate) / portf_vol_2

In [ ]:
#Transform to DataFrame
portf_res_2 = pd.DataFrame({'Returns': portf_rtns_2, 'Volatility': portf_vol_2, 'Sharpe Ratio': portf_sharpe_ratio_2})
portf_res_2

### 4.1.3 Filter 3

In [ ]:
n_assets_3 = len(portfolio_3)

weights_3 = np.random.random(size = (n_portfolios, n_assets_3))
weights_3 /= np.sum(weights_3, axis = 1)[:, np.newaxis]

cov_matrix_3 = daily_returns[portfolio_3].fillna(0).cov() * 252

In [ ]:
#Calculate the portfolio matrixs
portf_rtns_3 = np.dot(weights_3, avg_rtn_3)
portf_vol_3 = portf_vol(weights_3, cov_matrix_3)
portf_sharpe_ratio_3 = (portf_rtns_3 - interest_rate) / portf_vol_3

In [ ]:
#Transform to DataFrame
portf_res_3 = pd.DataFrame({'Returns': portf_rtns_3, 'Volatility': portf_vol_3, 'Sharpe Ratio': portf_sharpe_ratio_3})
portf_res_3

### 4.1.4 Filter 4

In [ ]:
n_assets_4 = len(portfolio_4)

weights_4 = np.random.random(size = (n_portfolios, n_assets_4))
weights_4 /= np.sum(weights_4, axis = 1)[:, np.newaxis]

cov_matrix_4 = daily_returns[portfolio_4].fillna(0).cov() * 252

In [ ]:
#Calculate the portfolio matrixs
portf_rtns_4 = np.dot(weights_4, avg_rtn_4)
portf_vol_4 = portf_vol(weights_4, cov_matrix_4)
portf_sharpe_ratio_4 = (portf_rtns_4 - interest_rate) / portf_vol_4

In [ ]:
#Transform to DataFrame
portf_res_4 = pd.DataFrame({'Returns': portf_rtns_4, 'Volatility': portf_vol_4, 'Sharpe Ratio': portf_sharpe_ratio_4})
portf_res_4

### 4.1.5 Filter 5

In [ ]:
n_assets_5 = len(portfolio_5)

weights_5 = np.random.random(size = (n_portfolios, n_assets_5))
weights_5 /= np.sum(weights_5, axis = 1)[:, np.newaxis]

cov_matrix_5 = daily_returns[portfolio_5].cov() * 252

In [ ]:
#Calculate the portfolio matrixs
portf_rtns_5 = np.dot(weights_5, avg_rtn_5)
portf_vol_5 = portf_vol(weights_5, cov_matrix_5)
portf_sharpe_ratio_5 = (portf_rtns_5 - interest_rate) / portf_vol_5

In [ ]:
#Transform to DataFrame
portf_res_5 = pd.DataFrame({'Returns': portf_rtns_5, 'Volatility': portf_vol_5, 'Sharpe Ratio': portf_sharpe_ratio_5})
portf_res_5

## 4.2 Monte Carlo Chart

### 4.2.1 Filter 1

In [ ]:
# Simulation Portfolio Scatter Chart
portf_1_plot = portf_res_1.hvplot.scatter(x = 'Volatility', y = 'Returns', c = "Sharpe Ratio", 
                                      height = 400, width = 800, title = 'Monte Carlo Simulation for Filter 1')
portf_1_plot

In [ ]:
# Plot the efficient frontier curve

ef_1 = ef_curve(portf_res_1, portf_rtns_1, portf_vol_1)
ef_1_plot = ef_1.hvplot.scatter(x = 'Volatility', y = 'Returns', title = 'Effecient Frointer for Filter 1')
ef_1_plot

In [ ]:
# Combine the charts
(portf_1_plot * ef_1_plot).opts(height = 400, width = 900)

### 4.2.2 Filter 2

In [ ]:
# Simulation Portfolio Scatter Chart
portf_2_plot = portf_res_2.hvplot.scatter(x = 'Volatility', y = 'Returns', c = "Sharpe Ratio", 
                                      height = 400, width = 800, title = 'Monte Carlo Simulation for Filter 2')
portf_2_plot

In [ ]:
# Plot the efficient frontier curve

ef_2 = ef_curve(portf_res_2, portf_rtns_2, portf_vol_2)
ef_2_plot = ef_2.hvplot.scatter(x = 'Volatility', y = 'Returns', title = 'Effecient Frointer for Filter 2')
ef_2_plot

In [ ]:
# Combine the charts
(portf_2_plot * ef_2_plot).opts(height = 400, width = 900)

### 4.2.3 Filter 3

In [ ]:
# Simulation Portfolio Scatter Chart
portf_3_plot = portf_res_3.hvplot.scatter(x = 'Volatility', y = 'Returns', c = "Sharpe Ratio", 
                                      height = 400, width = 800, title = 'Monte Carlo Simulation for Filter 3')
portf_3_plot

In [ ]:
# Plot the efficient frontier curve

ef_3 = ef_curve(portf_res_3, portf_rtns_3, portf_vol_3)
ef_3_plot = ef_3.hvplot.scatter(x = 'Volatility', y = 'Returns', title = 'Effecient Frointer for Filter 3')
ef_3_plot

In [ ]:
# Combine the charts
(portf_3_plot * ef_3_plot).opts(height = 400, width = 900)

### 4.2.4 Filter 4

In [ ]:
# Simulation Portfolio Scatter Chart
portf_4_plot = portf_res_4.hvplot.scatter(x = 'Volatility', y = 'Returns', c = "Sharpe Ratio", 
                                      height = 400, width = 800, title = 'Monte Carlo Simulation for Filter 4')
portf_4_plot

In [ ]:
# Plot the efficient frontier curve

ef_4 = ef_curve(portf_res_4, portf_rtns_4, portf_vol_4)
ef_4_plot = ef_4.hvplot.scatter(x = 'Volatility', y = 'Returns', title = 'Effecient Frointer for Filter 4')
ef_4_plot

In [ ]:
# Combine the charts
(portf_4_plot * ef_4_plot).opts(height = 400, width = 900)

### 4.2.5 Filter 5

In [ ]:
# Simulation Portfolio Scatter Chart
portf_5_plot = portf_res_5.hvplot.scatter(x = 'Volatility', y = 'Returns', c = "Sharpe Ratio", 
                                      height = 400, width = 800, title = 'Monte Carlo Simulation for Filter 5')
portf_5_plot

In [ ]:
# Plot the efficient frontier curve

ef_5 = ef_curve(portf_res_5, portf_rtns_5, portf_vol_5)
ef_5_plot = ef_5.hvplot.scatter(x = 'Volatility', y = 'Returns', title = 'Effecient Frointer for Filter 5')
ef_5_plot

In [ ]:
# Combine the charts
(portf_5_plot * ef_5_plot).opts(height = 400, width = 900)

## 4.3 Show the highest Sharpe Ratio Portfolio

### 4.3.1 Filter 1

In [ ]:
#Extract the max_sharpe_ratio
max_sharpe_ind_1 = np.argmax(portf_res_1['Sharpe Ratio'])
max_sharpe_portf_1 = portf_res_1.loc[max_sharpe_ind_1]

In [ ]:
# Show what is the optimized portfolio

#print_portfolio_summary(max_sharpe_portf_1, weights_1[max_sharpe_ind_1], portfolio_1, name = "Maximum Sharpe Ratio For Filter 1")

In [ ]:
# Show Pie Chart for the optimized portfolio

# data = zip(portfolio_1, weights_1[max_sharpe_ind_1])

# # Unpack the zipped data into separate lists
# labels, values = zip(*data)

# # Create a pie chart
# fig, ax = plt.subplots(figsize = (10, 6))
# ax.pie(values, labels = labels, autopct = '%1.2f%%', startangle = 50)
# ax.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.

# plt.title('Portfolio Composition For Filter 1')
# plt.show()

### 4.3.2 Filter 2

In [ ]:
#Extract the max_sharpe_ratio
max_sharpe_ind_2 = np.argmax(portf_res_2['Sharpe Ratio'])
max_sharpe_portf_2 = portf_res_2.loc[max_sharpe_ind_2]

In [ ]:
# Show what is the optimized portfolio

#print_portfolio_summary(max_sharpe_portf_2, weights_2[max_sharpe_ind_2], portfolio_2, name = "Maximum Sharpe Ratio For Filter 2")

In [ ]:
# Show Pie Chart for the optimized portfolio

# data = zip(portfolio_2, weights_2[max_sharpe_ind_2])

# # Unpack the zipped data into separate lists
# labels, values = zip(*data)

# # Create a pie chart
# fig, ax = plt.subplots(figsize = (10, 6))
# ax.pie(values, labels = labels, autopct = '%1.2f%%', startangle = 50)
# ax.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.

# plt.title('Portfolio Composition For Filter 2')
# plt.show()

### 4.3.3 Filter 3

In [ ]:
#Extract the max_sharpe_ratio
max_sharpe_ind_3 = np.argmax(portf_res_3['Sharpe Ratio'])
max_sharpe_portf_3 = portf_res_3.loc[max_sharpe_ind_3]

In [ ]:
# Show what is the optimized portfolio

#print_portfolio_summary(max_sharpe_portf_3, weights_3[max_sharpe_ind_3], portfolio_3, name = "Maximum Sharpe Ratio For Filter 3")

In [ ]:
# Show Pie Chart for the optimized portfolio

# data = zip(portfolio_3, weights_3[max_sharpe_ind_2])

# # Unpack the zipped data into separate lists
# labels, values = zip(*data)

# # Create a pie chart
# fig, ax = plt.subplots(figsize = (10, 6))
# ax.pie(values, labels = labels, autopct = '%1.2f%%', startangle = 50)
# ax.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.

# plt.title('Portfolio Composition For Filter 3')
# plt.show()

### 4.3.4 Filter 4

In [ ]:
#Extract the max_sharpe_ratio
max_sharpe_ind_4 = np.argmax(portf_res_4['Sharpe Ratio'])
max_sharpe_portf_4 = portf_res_4.loc[max_sharpe_ind_4]

In [ ]:
# Show what is the optimized portfolio

#print_portfolio_summary(max_sharpe_portf_4, weights_4[max_sharpe_ind_4], portfolio_4, name = "Maximum Sharpe Ratio For Filter 4")

In [ ]:
# Show Pie Chart for the optimized portfolio

# data = zip(portfolio_4, weights_4[max_sharpe_ind_4])

# # Unpack the zipped data into separate lists
# labels, values = zip(*data)

# # Create a pie chart
# fig, ax = plt.subplots(figsize = (10, 6))
# ax.pie(values, labels = labels, autopct = '%1.2f%%', startangle = 50)
# ax.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.

# plt.title('Portfolio Composition For Filter 4')
# plt.show()

### 4.3.5 Filter 5

In [ ]:
#Extract the max_sharpe_ratio
max_sharpe_ind_5 = np.argmax(portf_res_5['Sharpe Ratio'])
max_sharpe_portf_5 = portf_res_5.loc[max_sharpe_ind_5]

In [ ]:
# Show what is the optimized portfolio

print_portfolio_summary(max_sharpe_portf_5, weights_5[max_sharpe_ind_5], portfolio_5, name = "Maximum Sharpe Ratio For Filter 5")

In [ ]:
# Show Pie Chart for the optimized portfolio

# data = zip(portfolio_5, weights_5[max_sharpe_ind_5])

# # Unpack the zipped data into separate lists
# labels, values = zip(*data)

# # Create a pie chart
# fig, ax = plt.subplots(figsize = (10, 6))
# ax.pie(values, labels = labels, autopct = '%1.2f%%', startangle = 50)
# ax.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.

# plt.title('Portfolio Composition For Filter 5')
# plt.show()

# 5. Comparison with Portfolio and BTC

In [ ]:
# Calculate BTC cumulative return using log method

btc_cum_rtn = daily_returns['BTC'].cumsum().apply(np.exp)
btc_cum_rtn

In [ ]:
# Plot the cumulative return of BTC

btc_plot = btc_cum_rtn.hvplot(title = 'Cumulative Return of BTC', xlabel = 'Date', ylabel = 'Cumulative Return')
btc_plot

## 5.1 Filter 1

In [ ]:
# Create a optimized portfolio from the result of Monte Carlo Simulation

portfolio_rtn_1 = daily_returns[portfolio_1] * weights_1[max_sharpe_ind_1]
portfolio_rtn_1['Portfolio'] = portfolio_rtn_1.sum(axis = 1)
cum_port_rtn_1 = portfolio_rtn_1['Portfolio'].cumsum().apply(np.exp)
cum_port_rtn_1

In [ ]:
# Plot the cumulative return of portfolio

port_1_plot = cum_port_rtn_1.hvplot(title = 'Cumulative Return of Filter 1', xlabel = 'Date', ylabel = 'Cumulative Return')
(port_1_plot * btc_plot).opts(title = 'Cumulative Returns Comparison with Filter 1 and BTC')

## 5.2 Filter 2

In [ ]:
# Create a optimized portfolio from the result of Monte Carlo Simulation

portfolio_rtn_2 = daily_returns[portfolio_2] * weights_2[max_sharpe_ind_2]
portfolio_rtn_2['Portfolio'] = portfolio_rtn_2.sum(axis = 1)
cum_port_rtn_2 = portfolio_rtn_2['Portfolio'].cumsum().apply(np.exp)
cum_port_rtn_2

In [ ]:
# Plot the cumulative return of portfolio

port_2_plot = cum_port_rtn_2.hvplot(title = 'Cumulative Return of Filter 2', xlabel = 'Date', ylabel = 'Cumulative Return')
(port_2_plot * btc_plot).opts(title = 'Cumulative Returns Comparison with Filter 2 and BTC')

## 5.3 Filter 3

In [ ]:
# Create a optimized portfolio from the result of Monte Carlo Simulation

portfolio_rtn_3 = daily_returns[portfolio_3] * weights_3[max_sharpe_ind_3]
portfolio_rtn_3['Portfolio'] = portfolio_rtn_3.sum(axis = 1)
cum_port_rtn_3 = portfolio_rtn_3['Portfolio'].cumsum().apply(np.exp)
cum_port_rtn_3

In [ ]:
# Plot the cumulative return of portfolio

port_3_plot = cum_port_rtn_3.hvplot(title = 'Cumulative Return of Filter 3', xlabel = 'Date', ylabel = 'Cumulative Return')
(port_3_plot * btc_plot).opts(title = 'Cumulative Returns Comparison with Filter 3 and BTC')

## 5.4 Filter 4

In [ ]:
# Create a optimized portfolio from the result of Monte Carlo Simulation

portfolio_rtn_4 = daily_returns[portfolio_4] * weights_4[max_sharpe_ind_4]
portfolio_rtn_4['Portfolio'] = portfolio_rtn_4.sum(axis = 1)
cum_port_rtn_4 = portfolio_rtn_4['Portfolio'].cumsum().apply(np.exp)
cum_port_rtn_4

In [ ]:
# Plot the cumulative return of portfolio

port_4_plot = cum_port_rtn_4.hvplot(title = 'Cumulative Return of Filter 4', xlabel = 'Date', ylabel = 'Cumulative Return')
(port_4_plot * btc_plot).opts(title = 'Cumulative Returns Comparison with Filter 4 and BTC')

## 5.5 Filter 5

In [ ]:
# Create a optimized portfolio from the result of Monte Carlo Simulation

portfolio_rtn_5 = daily_returns[portfolio_5] * weights_5[max_sharpe_ind_5]
portfolio_rtn_5['Portfolio'] = portfolio_rtn_5.sum(axis = 1)
cum_port_rtn_5 = portfolio_rtn_5['Portfolio'].cumsum().apply(np.exp)
cum_port_rtn_5

In [ ]:
# Plot the cumulative return of portfolio

port_5_plot = cum_port_rtn_5.hvplot(title = 'Cumulative Return of Filter 5', xlabel = 'Date', ylabel = 'Cumulative Return')
(port_5_plot * btc_plot).opts(title = 'Cumulative Returns Comparison with Filter 5 and BTC')

# 6. Backtesing Trading Strategy

## 6.1 Trading strategy 1: Crossover Moving Average
* When portfolio price is above its own moving average curve, long the position
* When portfolio price is lower its own moving average curve, close the position

In [ ]:
"""
    Helper function to run different days moving average strategy.

    Args:
        days (int): How many days for the moving average calculation
        df_price (DataFrame): Stocks prices DataFrame
        df_rtn (DataFrame): Stocks portfolio return DataFrame        
"""
def strategy_ma(days, df_price, df_rtn):
    
    if days < 21:
        df_price[f"ema_{days}"] = df_price['Portfolio'].ewm(span = days, adjust = False, min_periods = days).mean()
        
        df_price[f"pos_ema_{days}"] = (df_price['Portfolio'] > df_price[f"ema_{days}"]).astype(int)
        
        nums_trades = sum((df_price[f"pos_ema_{days}"] == 1) & (df_price[f"pos_ema_{days}"].shift(1) == 0))
        df_price[f"ema_{days}_rtn"] = df_price[f"pos_ema_{days}"].shift(1) * df_rtn
        df_price[f"ema_{days}_cum_rtn"] = df_price[f"ema_{days}_rtn"].cumsum().apply(np.exp)
        
        print(f"{nums_trades} trades in this strategy")
        
        return df_price[f"ema_{days}_cum_rtn"].hvplot(ylabel = 'Cumulative Return', 
                                                      title = f"Cumulative of EMA {days} Strategy Return")
    
    
    else:
        df_price[f"ma_{days}"] = df_price['Portfolio'].rolling(window = days).mean()
        df_price[f"pos_ma_{days}"] = (df_price['Portfolio'] > df_price[f"ma_{days}"]).astype(int)
        nums_trades = sum((df_price[f"pos_ma_{days}"] == 1) & (df_price[f"pos_ma_{days}"].shift(1) == 0))
        df_price[f"ma_{days}_rtn"] = df_price[f"pos_ma_{days}"].shift(1) * df_rtn
        df_price[f"ma_{days}_cum_rtn"] = df_price[f"ma_{days}_rtn"].cumsum().apply(np.exp)
        
        print(f"{nums_trades} trades in this strategy")
        
        return df_price[f"ma_{days}_cum_rtn"].hvplot(ylabel = 'Cumulative Return', 
                                                      title = f"Cumulative of MA {days} Strategy Return")
    

### 6.1.1 Different MA Performance Analysis

#### 6.1.1.1 Filter 1

In [ ]:
# Get the portfolio's close price

portfolio_prices_1 = stocks_close[portfolio_1] * weights_1[max_sharpe_ind_1]
portfolio_prices_1['Portfolio'] = portfolio_prices_1.sum(axis = 1)

In [ ]:
f1_ema_8_plot = strategy_ma(8, portfolio_prices_1, portfolio_rtn_1['Portfolio'])
f1_ema_8_plot

In [ ]:
f1_ema_20_plot = strategy_ma(20, portfolio_prices_1, portfolio_rtn_1['Portfolio'])
f1_ema_20_plot

In [ ]:
f1_ma_50_plot = strategy_ma(50, portfolio_prices_1, portfolio_rtn_1['Portfolio'])
f1_ma_50_plot

In [ ]:
f1_ma_200_plot = strategy_ma(200, portfolio_prices_1, portfolio_rtn_1['Portfolio'])
f1_ma_200_plot

In [ ]:
(port_1_plot * btc_plot * f1_ema_8_plot * f1_ema_20_plot * f1_ma_50_plot * f1_ma_200_plot).opts(title = 'Performance Comparison with Filter 1 and BTC', 
                                                                               width = 900, height = 400, legend_position = 'bottom_left')

#### 6.1.1.2 Filter 2

In [ ]:
# Get the portfolio's close price

portfolio_prices_2 = stocks_close[portfolio_2] * weights_2[max_sharpe_ind_2]
portfolio_prices_2['Portfolio'] = portfolio_prices_2.sum(axis = 1)

In [ ]:
f2_ema_8_plot = strategy_ma(8, portfolio_prices_2, portfolio_rtn_2['Portfolio'])
f2_ema_8_plot

In [ ]:
f2_ema_20_plot = strategy_ma(20, portfolio_prices_2, portfolio_rtn_2['Portfolio'])
f2_ema_20_plot

In [ ]:
f2_ma_50_plot = strategy_ma(50, portfolio_prices_2, portfolio_rtn_2['Portfolio'])
f2_ma_50_plot

In [ ]:
f2_ma_200_plot = strategy_ma(200, portfolio_prices_2, portfolio_rtn_2['Portfolio'])
f2_ma_200_plot

In [ ]:
(port_2_plot * btc_plot * f2_ema_8_plot * f2_ema_20_plot * f2_ma_50_plot * f2_ma_200_plot).opts(title = 'Performance Comparison with Filter 2 and BTC', 
                                                                               width = 900, height = 400, legend_position = 'bottom_left')

#### 6.1.1.3 Filter 3

In [ ]:
# Get the portfolio's close price

portfolio_prices_3 = stocks_close[portfolio_3] * weights_3[max_sharpe_ind_3]
portfolio_prices_3['Portfolio'] = portfolio_prices_3.sum(axis = 1)

In [ ]:
f3_ema_8_plot = strategy_ma(8, portfolio_prices_3, portfolio_rtn_3['Portfolio'])
f3_ema_8_plot

In [ ]:
f3_ema_20_plot = strategy_ma(20, portfolio_prices_3, portfolio_rtn_3['Portfolio'])
f3_ema_20_plot

In [ ]:
f3_ma_50_plot = strategy_ma(50, portfolio_prices_3, portfolio_rtn_3['Portfolio'])
f3_ma_50_plot

In [ ]:
f3_ma_200_plot = strategy_ma(200, portfolio_prices_3, portfolio_rtn_3['Portfolio'])
f3_ma_200_plot

In [ ]:
(port_3_plot * btc_plot * f3_ema_8_plot * f3_ema_20_plot * f3_ma_50_plot * f3_ma_200_plot).opts(title = 'Performance Comparison with Filter 3 and BTC', 
                                                                               width = 900, height = 400, legend_position = 'bottom_left')

#### 6.1.1.4 Filter 5

In [ ]:
# Get the portfolio's close price

portfolio_prices_5 = stocks_close[portfolio_5] * weights_5[max_sharpe_ind_5]
portfolio_prices_5['Portfolio'] = portfolio_prices_5.sum(axis = 1)

In [ ]:
f5_ema_8_plot = strategy_ma(8, portfolio_prices_5, portfolio_rtn_5['Portfolio'])
f5_ema_8_plot

In [ ]:
f5_ema_20_plot = strategy_ma(20, portfolio_prices_5, portfolio_rtn_5['Portfolio'])
f5_ema_20_plot

In [ ]:
f5_ma_50_plot = strategy_ma(50, portfolio_prices_5, portfolio_rtn_5['Portfolio'])
f5_ma_50_plot

In [ ]:
f5_ma_200_plot = strategy_ma(200, portfolio_prices_5, portfolio_rtn_5['Portfolio'])
f5_ma_200_plot

In [ ]:
(port_5_plot * btc_plot * f5_ema_8_plot * f5_ema_20_plot * f5_ma_50_plot * f5_ma_200_plot).opts(title = 'Performance Comparison with Filter 5 and BTC', 
                                                                               width = 900, height = 400, legend_position = 'top_left')

## 6.2 Trading strategy 2: (Hypothesis: BTC is the leading indicator)
* When BTC price is above its moving average curve, long the position
* When BTC price is lower its moving average curve, close the position

In [ ]:
"""
    Helper function to run different days moving average strategy for BTC leading indicator.

    Args:
        days (int): How many days for the moving average calculation
        df_price (DataFrame): Stocks prices DataFrame
        df_rtn (DataFrame): Stocks portfolio return DataFrame
        btc_df (DataFrame): BTC price DataFrame
"""
def strategy_ma_2(days, df_price, df_rtn, btc_df):
    
    
    
    if days < 21:
        df_price[f"ema_{days}"] = df_price['Portfolio'].ewm(span = days, adjust = False, min_periods = days).mean()
        
        btc_df[f"ema_{days}"] = btc_df['BTC'].ewm(span = days, adjust = False, min_periods = days).mean()
        
        df_price[f"pos_ema_{days}_2"] = (btc_df['BTC'] > btc_df[f"ema_{days}"]).astype(int)
        
        nums_trades = sum(
           
            (df_price[f"pos_ema_{days}_2"] == 1) & (df_price[f"pos_ema_{days}_2"].shift(1) == 0)
        )
        
        df_price[f"ema_{days}_rtn_2"] = df_price[f"pos_ema_{days}_2"].shift(1) * df_rtn
        df_price[f"ema_{days}_cum_rtn_2"] = df_price[f"ema_{days}_rtn_2"].cumsum().apply(np.exp)
        
        print(f"{nums_trades} trades in this strategy")
        
        return df_price[f"ema_{days}_cum_rtn_2"].hvplot(ylabel = 'Cumulative Return', 
                                                      title = f"Cumulative of EMA {days} Strategy Return Under BTC Leading")
    
    
    else:
        df_price[f"ma_{days}_2"] = df_price['Portfolio'].rolling(window = days).mean()
        
        btc_df[f"ma_{days}"] = btc_df['BTC'].rolling(window = 20).mean()
        df_price[f"pos_ma_{days}_2"] = (btc_df['BTC'] > btc_df[f"ma_{days}"]).astype(int)
        nums_trades = sum((df_price[f"pos_ma_{days}_2"] == 1) & (df_price[f"pos_ma_{days}_2"].shift(1) == 0))
        df_price[f"ma_{days}_rtn_2"] = df_price[f"pos_ma_{days}_2"].shift(1) * df_rtn
        df_price[f"ma_{days}_cum_rtn_2"] = df_price[f"ma_{days}_rtn_2"].cumsum().apply(np.exp)
        
        print(f"{nums_trades} trades in this strategy")
        
        return df_price[f"ma_{days}_cum_rtn_2"].hvplot(ylabel = 'Cumulative Return', 
                                                      title = f"Cumulative of MA {days} Strategy Return Under BTC Leading")

In [ ]:
# Create the BTC SMA 20 curve data

btc_close['sma_20'] = btc_close.rolling(window = 20).mean()
btc_close['sma_20']

In [ ]:
# Create trading signal as position

portfolio_prices['position_btc_leading'] = (btc_close['BTC'] > btc_close['sma_20']).astype(int)
portfolio_prices['position_btc_leading']

In [ ]:
# Calculate the strategy return

portfolio_prices['strategy_2_rtn'] = portfolio_prices['position_btc_leading'].shift(1) * portfolio_rtn['Portfolio']
portfolio_prices['strategy_2_rtn']

In [ ]:
# Calculate the cumulative return of the strategy

portfolio_prices['strategy_2_cum_rtn'] = portfolio_prices['strategy_2_rtn'].cumsum().apply(np.exp)

In [ ]:
# Plot the cumulative return of strategy

strategy_2_plot = portfolio_prices['strategy_2_cum_rtn'].hvplot(ylabel = 'Cumulative Return', 
                                                            title = 'Cumulative Strategy Return')
strategy_2_plot

In [ ]:
#Combine the plots of portfolio, BTC and strategys return

(port_plot * btc_plot * strategy_1_plot * strategy_2_plot).opts(title = 'Comparison with Portfolio, BTC and Strategys Return', 
                                                                 width = 900, height = 400, legend_position = 'top_left')

In [ ]:
# Show how many times the portfolio trades over time

sum((portfolio_prices['position_btc_leading'] == 1) & (portfolio_prices['position_btc_leading'].shift(1) == 0))

In [ ]:
stocks_daily['Volume'].hvplot.bar()

### 6.2.1 Different MA Performance Analysis (BTC Leading Indicator)

#### 6.2.1.1 Filter 1

In [ ]:
f1_ema_20_2_plot = strategy_ma_2(20, portfolio_prices_1, portfolio_rtn_1['Portfolio'], btc_close)
f1_ema_20_2_plot

In [ ]:
f1_ema_8_2_plot = strategy_ma_2(8, portfolio_prices_1, portfolio_rtn_1['Portfolio'], btc_close)
f1_ema_8_2_plot

In [ ]:
f1_ma_50_2_plot = strategy_ma_2(50, portfolio_prices_1, portfolio_rtn_1['Portfolio'], btc_close)
f1_ma_50_2_plot

In [ ]:
f1_ma_200_2_plot = strategy_ma_2(200, portfolio_prices_1, portfolio_rtn_1['Portfolio'], btc_close)
f1_ma_200_2_plot

In [ ]:
(f1_ema_8_plot * f1_ema_8_2_plot).opts(title = 'Comparison with Strategies EMA 8 For Filter 1', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f1_ema_20_plot * f1_ema_20_2_plot).opts(title = 'Comparison with Strategies EMA 20 For Filter 1', 
                                         width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f1_ma_50_plot * f1_ma_50_2_plot).opts(title = 'Comparison with Strategies MA50 For Filter 1', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f1_ma_200_plot * f1_ma_200_2_plot).opts(title = 'Comparison with Strategies MA200 For Filter 1', 
                                         width = 900, height = 400, legend_position = 'bottom_left')

#### 6.2.1.2 Filter 2

In [ ]:
f2_ema_8_2_plot = strategy_ma_2(8, portfolio_prices_2, portfolio_rtn_2['Portfolio'], btc_close)
f2_ema_8_2_plot

In [ ]:
f2_ema_20_2_plot = strategy_ma_2(20, portfolio_prices_2, portfolio_rtn_2['Portfolio'], btc_close)
f2_ema_20_2_plot

In [ ]:
f2_ma_50_2_plot = strategy_ma_2(50, portfolio_prices_2, portfolio_rtn_2['Portfolio'], btc_close)
f2_ma_50_2_plot

In [ ]:
f2_ma_200_2_plot = strategy_ma_2(200, portfolio_prices_2, portfolio_rtn_2['Portfolio'], btc_close)
f2_ma_200_2_plot

In [ ]:
(f2_ema_8_plot * f2_ema_8_2_plot).opts(title = 'Comparison with Strategies EMA 8 For Filter 2', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f2_ema_20_plot * f2_ema_20_2_plot).opts(title = 'Comparison with Strategies EMA 20 For Filter 2', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f2_ma_50_plot * f2_ma_50_2_plot).opts(title = 'Comparison with Strategies MA50 For Filter 2', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f2_ma_200_plot * f2_ma_200_2_plot).opts(title = 'Comparison with Strategies MA200 For Filter 2', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

#### 6.2.1.3 Filter 3

In [ ]:
f3_ema_8_2_plot = strategy_ma_2(8, portfolio_prices_3, portfolio_rtn_3['Portfolio'], btc_close)
f3_ema_8_2_plot

In [ ]:
f3_ema_20_2_plot = strategy_ma_2(20, portfolio_prices_3, portfolio_rtn_3['Portfolio'], btc_close)
f3_ema_20_2_plot

In [ ]:
f3_ma_50_2_plot = strategy_ma_2(50, portfolio_prices_3, portfolio_rtn_3['Portfolio'], btc_close)
f3_ma_50_2_plot

In [ ]:
f3_ma_200_2_plot = strategy_ma_2(200, portfolio_prices_3, portfolio_rtn_3['Portfolio'], btc_close)
f3_ma_200_2_plot

In [ ]:
(f3_ema_8_plot * f3_ema_8_2_plot).opts(title = 'Comparison with Strategies EMA 8 For Filter 3', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f3_ema_20_plot * f3_ema_20_2_plot).opts(title = 'Comparison with Strategies EMA 20 For Filter 3', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f3_ma_50_plot * f3_ma_50_2_plot).opts(title = 'Comparison with Strategies MA50 For Filter 3', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

In [ ]:
(f3_ma_200_plot * f3_ma_200_2_plot).opts(title = 'Comparison with Strategies MA200 For Filter 3', 
                                       width = 900, height = 400, legend_position = 'bottom_left')

#### 6.2.1.4 Filter 5

In [ ]:
f5_ema_8_2_plot = strategy_ma_2(8, portfolio_prices_5, portfolio_rtn_5['Portfolio'], btc_close)
f5_ema_8_2_plot

In [ ]:
f5_ema_20_2_plot = strategy_ma_2(20, portfolio_prices_5, portfolio_rtn_5['Portfolio'], btc_close)
f5_ema_20_2_plot

In [ ]:
f5_ma_50_2_plot = strategy_ma_2(50, portfolio_prices_5, portfolio_rtn_5['Portfolio'], btc_close)
f5_ma_50_2_plot

In [ ]:
f5_ma_200_2_plot = strategy_ma_2(200, portfolio_prices_5, portfolio_rtn_5['Portfolio'], btc_close)
f5_ma_200_2_plot

In [ ]:
(f5_ema_8_plot * f5_ema_8_2_plot).opts(title = 'Comparison with Strategies EMA 8 For Filter 5', 
                                       width = 900, height = 400, legend_position = 'top_left')

In [ ]:
(f5_ema_20_plot * f5_ema_20_2_plot).opts(title = 'Comparison with Strategies EMA 20 For Filter 5', 
                                         width = 900, height = 400, legend_position = 'top_left')

In [ ]:
(f5_ma_50_plot * f5_ma_50_2_plot).opts(title = 'Comparison with Strategies MA50 For Filter 5', 
                                       width = 900, height = 400, legend_position = 'top_left')

In [ ]:
(f5_ma_200_plot * f5_ma_200_2_plot).opts(title = 'Comparison with Strategies MA200 For Filter 5', 
                                         width = 900, height = 400, legend_position = 'top_left')